In [1]:
import json
from glob import glob
import fnmatch
from multiprocessing import Pool
from pathlib import Path
import os, itertools
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from pickle import dumps, loads, dump, load

from stexls import *

In [2]:
%rm /tmp/linker.bin

rm: cannot remove '/tmp/linker.bin': No such file or directory


In [3]:
try:
    with open('/tmp/linker.bin', 'rb') as fd:
        linker = load(fd)
except Exception as e:
    print(e)
    root = Path.home()/'MathHub'
    linker = Linker(root=root, file_pattern='smglom/**/*.tex')
    print(linker.update(progress=tqdm, use_multiprocessing=True))
    with open('/tmp/linker.bin', 'w+b') as fd:
        dump(linker, fd)

[Errno 2] No such file or directory: '/tmp/linker.bin'


/home/marian/MathHub/smglom/IWGS/source/jupyterNB.tex:2:3: File not indexed:"/home/marian/MathHub/MiKoMH/TDM/source/digdocs/en/markdown.tex"
/home/marian/MathHub/smglom/IWGS/source/jupyterNB.tex:3:3: File not indexed:"/home/marian/MathHub/MiKoMH/TDM/source/LaTeX/en/tex-packages.tex"
/home/marian/MathHub/smglom/chevahir/source/maximaltorus.tex:2:1: File not indexed:"/home/marian/MathHub/smglom/algebra/source/algebraictorus.tex"

{}


In [4]:
%ls -ahl /tmp/linker.bin

-rw-rw-r-- 1 marian marian 23M Apr  7 15:27 /tmp/linker.bin


In [5]:
path = root/ Path('smglom/primes/source/balancedprime.en.tex')

In [6]:
linker.view_import_graph(path)

In [7]:
linker.lazy_build_order_update = True

In [8]:
linker.update(tqdm)

{}

In [9]:
for object in linker.objects[path]:
    print(linker.links[object].format())
    print()
    print('-'*80)
    print()

Contains files:
	/home/marian/MathHub/smglom/arithmetics/source/naturalnumbers.tex
	/home/marian/MathHub/smglom/sets/source/symmetric.tex
	/home/marian/MathHub/smglom/sets/source/partial-order.tex
	/home/marian/MathHub/smglom/sets/source/reflexive.tex
	/home/marian/MathHub/smglom/mv/source/argseq.tex
	/home/marian/MathHub/smglom/sets/source/quotientspace.tex
	/home/marian/MathHub/smglom/sets/source/total-relation.tex
	/home/marian/MathHub/smglom/arithmetics/source/natarith.tex
	/home/marian/MathHub/smglom/mv/source/expression.tex
	/home/marian/MathHub/smglom/primes/source/balancedprime.en.tex
	/home/marian/MathHub/smglom/mv/source/piecewise.tex
	/home/marian/MathHub/smglom/arithmetics/source/divisor.tex
	/home/marian/MathHub/smglom/sets/source/equivalence-relation.tex
	/home/marian/MathHub/smglom/arithmetics/source/intarith.tex
	/home/marian/MathHub/smglom/sets/source/union.tex
	/home/marian/MathHub/smglom/sets/source/preorder.tex
	/home/marian/MathHub/smglom/arithmetics/source/rationa